In [1]:
import numpy as np
import GPy
import seaborn as sns
import math
import matplotlib.pyplot as plt
import edward as ed
import tensorflow as tf
import tensorflow.contrib.eager as tfe
tfe.enable_eager_execution()
from edward.models import MultivariateNormalFullCovariance, Normal
from scipy.stats import bernoulli
from tensorflow.contrib.distributions import  Bernoulli
from tensorflow.contrib.distributions import Categorical

In [2]:
rate = 100
sess = tf.Session()
N = np.random.poisson(rate)
S = np.expand_dims(np.linspace(0.0,1.0,N), axis=1)
print(S)
# print(sess.run(S).shape)

[[ 0.        ]
 [ 0.01052632]
 [ 0.02105263]
 [ 0.03157895]
 [ 0.04210526]
 [ 0.05263158]
 [ 0.06315789]
 [ 0.07368421]
 [ 0.08421053]
 [ 0.09473684]
 [ 0.10526316]
 [ 0.11578947]
 [ 0.12631579]
 [ 0.13684211]
 [ 0.14736842]
 [ 0.15789474]
 [ 0.16842105]
 [ 0.17894737]
 [ 0.18947368]
 [ 0.2       ]
 [ 0.21052632]
 [ 0.22105263]
 [ 0.23157895]
 [ 0.24210526]
 [ 0.25263158]
 [ 0.26315789]
 [ 0.27368421]
 [ 0.28421053]
 [ 0.29473684]
 [ 0.30526316]
 [ 0.31578947]
 [ 0.32631579]
 [ 0.33684211]
 [ 0.34736842]
 [ 0.35789474]
 [ 0.36842105]
 [ 0.37894737]
 [ 0.38947368]
 [ 0.4       ]
 [ 0.41052632]
 [ 0.42105263]
 [ 0.43157895]
 [ 0.44210526]
 [ 0.45263158]
 [ 0.46315789]
 [ 0.47368421]
 [ 0.48421053]
 [ 0.49473684]
 [ 0.50526316]
 [ 0.51578947]
 [ 0.52631579]
 [ 0.53684211]
 [ 0.54736842]
 [ 0.55789474]
 [ 0.56842105]
 [ 0.57894737]
 [ 0.58947368]
 [ 0.6       ]
 [ 0.61052632]
 [ 0.62105263]
 [ 0.63157895]
 [ 0.64210526]
 [ 0.65263158]
 [ 0.66315789]
 [ 0.67368421]
 [ 0.68421053]
 [ 0.69473

In [3]:
k = GPy.kern.RBF(input_dim=1,lengthscale=0.1)
mu = np.zeros(N) # vector of the means
#print(sess.run(mu))

C = k.K(S, S)# covariance matrix
Z = np.random.multivariate_normal(mu,C)

In [4]:
S_tf = tf.constant(S, dtype=tf.float32)
Z_tf = tf.constant(Z, dtype=tf.float32)
V = 1 / (1 + tf.exp(-Z_tf))
R = tf.random_uniform([N])
accept = tf.where(R < V)
S_k = tf.squeeze(tf.gather(S_tf, accept), axis=-1)
Z_k = tf.gather(Z_tf, accept)

In [5]:
def conditional(x_new, x, y, kernel):

    B = kernel.K(x, x_new)
    A = kernel.K(x_new, x_new)
    C = kernel.K(x, x)
    N = tf.shape(C)[0]
    mu = tf.matmul(B, tf.matmul(tf.matrix_inverse(C + 1e-5*tf.eye(N)), y), transpose_a = True)
    sigma = A - tf.matmul(B, tf.matmul(tf.matrix_inverse(C + 1e-5*tf.eye(N)),B), transpose_a = True)
    return tf.squeeze(mu), tf.squeeze(sigma)

In [6]:
class RBF():
    
    def __init__(self, input_dim, variance=1., lengthscale=1.):
        self.input_dim = input_dim
        self.variance = variance
        self.lengthscale = lengthscale
        
    def K(self, X, X2):
        return self.variance * tf.exp(-self.square_dist(X, X2) / 2)
        
    def square_dist(self, X, X2):
        X = X / self.lengthscale
        Xs = tf.reduce_sum(tf.square(X), 1)
        if X2 is None:
            return -2 * tf.matmul(X, tf.transpose(X)) + tf.reshape(Xs, (-1, 1)) + tf.reshape(Xs, (1, -1))
        else:
            X2 = X2 / self.lengthscale
            X2s = tf.reduce_sum(tf.square(X2), 1)
            return -2 * tf.matmul(X, tf.transpose(X2)) + tf.reshape(Xs, (-1, 1)) + tf.reshape(X2s, (1, -1))

In [41]:
x_new = tf.random_uniform((1,1))
kern = RBF(input_dim=1, lengthscale=0.1)
mu_new, sigma_new = conditional(x_new, S_k, Z_k, kern)
print(x_K)

tf.Tensor(
[[ 0.01052632]
 [ 0.03157895]
 [ 0.04210526]
 [ 0.06315789]
 [ 0.09473684]
 [ 0.10526316]
 [ 0.11578947]
 [ 0.12631579]
 [ 0.15789473]
 [ 0.18947369]
 [ 0.2       ]
 [ 0.21052632]
 [ 0.23157895]
 [ 0.25263157]
 [ 0.2631579 ]
 [ 0.28421053]
 [ 0.29473683]
 [ 0.30526316]
 [ 0.31578946]
 [ 0.33684212]
 [ 0.36842105]
 [ 0.38947368]
 [ 0.40000001]
 [ 0.41052631]
 [ 0.42105263]
 [ 0.44210526]
 [ 0.45263159]
 [ 0.47368422]
 [ 0.48421052]
 [ 0.49473685]
 [ 0.52631581]
 [ 0.53684211]
 [ 0.55789477]
 [ 0.60000002]
 [ 0.62105262]
 [ 0.70526314]
 [ 0.7263158 ]
 [ 0.75789475]
 [ 0.77894735]
 [ 0.82105261]
 [ 0.83157897]
 [ 0.86315787]
 [ 0.87368423]
 [ 0.89473683]
 [ 0.92631578]
 [ 0.93684208]
 [ 0.95789474]
 [ 0.96842104]
 [ 0.9894737 ]
 [ 1.        ]], shape=(50, 1), dtype=float32)


In [ ]:
def add_event(x_new, y_new, x_M, y_M):
    x_M = tf.concat([x_M, x_new], 0)
    y_M = tf.concat([y_M, y_new], 0)
    return x_M, y_M
    
def erase_event(x_M, y_M, c):
    M = tf.shape(x_M)[0]
    mask = np.ones_like(x_M)
    mask[c] = 0
    x_M = tf.expand_dims(tf.boolean_mask(x_M, mask), axis=-1)
    y_M = tf.expand_dims(tf.boolean_mask(y_M, mask), axis=-1)
    
    return x_M, y_M

def insert_event(x_K, y_K, x_M, y_M):

    print("Event inserted")
    M = tf.shape(x_M)[0]
    x_new = tf.random_uniform((1,1))
    mu_new, sigma_new = conditional(x_new, tf.concat([x_K,x_M], 0), tf.concat([y_K, y_M], 0), kern)
    y_new = tf.random_normal((1,1), mean=mu_new, stddev=tf.sqrt(sigma_new))
    ratio = tf.log(float(rate))
    ratio -= tf.log(tf.cast(M+1, tf.float32))
    ratio -= tf.log(1+tf.exp(y_new))
    a = tf.random_uniform((1,))
    #print(tf.squeeze(tf.less(tf.log(a), ratio)))
    x_M, y_M = tf.cond(tf.squeeze(tf.less(tf.log(a), ratio)), lambda: add_event(x_new, y_new, x_M, y_M), lambda: (x_M, y_M))
    return x_M, y_M

def delete_event(x_K, y_K, x_M, y_M):
    print("Event deleted")
    M = tf.shape(x_M)[0]
    #print(M)
    if tf.equal(M, tf.constant(0)):
        return x_M, y_M
    c = np.random.choice(M, 1)[0]
    #print(c)
    ratio = tf.log(tf.cast(M, tf.float32))
    ratio += tf.log(1 + tf.exp(y_M[c]))
    ratio -= tf.log(float(rate))
    a = tf.random_uniform((1,))
    
    x_M, y_M = tf.cond(tf.squeeze(tf.less(tf.log(a), ratio)), lambda: erase_event(x_M, y_M, c), lambda: (x_M, y_M))

    return x_M, y_M


In [ ]:
def sample_cond(x_K, y_K, x_M, y_M, i):
    return tf.less(i, tf.shape(x_M)[0])

### Change location at a particular index
def sample_step(x_K, y_K, x_M, y_M, i):
    x_new = tf.random_uniform((1,1))
    mu_new, sigma_new = conditional(x_new, tf.concat([x_K,x_M], 0), tf.concat([y_K, y_M], 0), kern)
    y_new = tf.random_normal((1,1), mean=mu_new, stddev=tf.sqrt(sigma_new))
    ratio = tf.log(1 + tf.exp(y_M[i]))
    ratio -= tf.log(1 + tf.exp(y_new))
    a = tf.random_uniform((1,))
    x_M[i], y_M[i] = tf.cond(tf.less(tf.log(a), ratio), lambda: x_new, y_new, lambda: x_M[i], y_M[i])
    i = tf.add(i, 1)
    return x_K, y_K, x_M, y_M, i
    
def thinned_cond(x_K, y_K, x_M, y_M, i):
    return tf.less(i, tf.constant(10))

def thinned_step(x_K, y_K, x_M, y_M, i):
    
    bern = Bernoulli(probs = bern_p)
    x_M, y_M = tf.cond(tf.equal(bern.sample(), 1), lambda: insert_event(x_K, y_K, x_M, y_M), lambda: delete_event(x_K, y_K, x_M, y_M))
    tf.add(i, 1)
    return x_K, y_K, x_M, y_M, i

def loop_cond(n_iter,it, x_K, y_K, x_M, y_M):
    return tf.less(it, n_iter)

## Loop step
def loop_step(n_iter, it, x_K, y_K, x_M, y_M):
    
    # Sample number of thinned events
    i = tfe.Variable(0)
    res = tf.while_loop(thinned_cond, thinned_step, [x_K, y_K, x_M, y_M, i])
    
    # Sample thinned locations
    it = tfe.Variable(0)
    res = tf.while_loop(sample_cond, sample_step, [x_K, y_K, x_M, y_M, it])
    
    tf.add(it, 1)
    return n_iter, it, x_K, y_K, x_M, y_M
    
    
bern_p = 0.5
n_iter = tf.constant(30)
x_K, y_K = S_k, Z_k
x_M, y_M = tf.zeros((0,1)), tf.zeros((0,1))

it = tfe.Variable(0)
res = tf.while_loop(loop_cond, loop_step, [n_iter,it, x_K, y_K, x_M, y_M])
